In [1]:
!pip install firebase-admin
import firebase_admin
from firebase_admin import credentials, db
from datetime import datetime  # Untuk konversi timestamp ke human-readable
import pandas as pd


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Konfigurasi database sumber
source_cred = credentials.Certificate("/work/database-sensor-iklim-litbang-firebase-admin.json")
firebase_admin.initialize_app(source_cred, {
    'databaseURL': 'https://database-sensor-iklim-litbang-default-rtdb.asia-southeast1.firebasedatabase.app/'
})

In [3]:
# Referensi ke data sumber
source_ref = db.reference('/auto_weather_stat/id-03/data')

# Mengambil data dari database sumber
source_data = source_ref.get()

# Mengubah data menjadi DataFrame pandas
if source_data:
    # Mengubah data menjadi DataFrame
    df = pd.DataFrame.from_dict(source_data, orient='index')
    
    # Jika 'timestamp' sudah ada, jangan pindahkan indeks ke kolom
    if 'timestamp' not in df.columns:
        df.index.name = 'timestamp'  # Mengatur nama indeks
        df.reset_index(inplace=True)  # Memindahkan indeks menjadi kolom biasa
    
    print(df.head())  # Menampilkan data untuk verifikasi
else:
    print("Tidak ada data di jalur /auto_weather_stat/id-03/data.")

                 dew  humidity  pressure  temperature   timestamp  volt  humi  \
1731000619  24.83318     98.48   1011.58        25.09  1731000619  4.05   NaN   
1731000678  24.83488     98.49   1011.57        25.09  1731000678  4.05   NaN   
1731000739  24.83318     98.48   1011.56        25.09  1731000739  4.05   NaN   
1731000798  24.83148     98.47   1011.51        25.09  1731000798  4.05   NaN   
1731000859  24.79155     98.47   1011.54        25.05  1731000859  4.05   NaN   

            pres  temp  
1731000619   NaN   NaN  
1731000678   NaN   NaN  
1731000739   NaN   NaN  
1731000798   NaN   NaN  
1731000859   NaN   NaN  


In [4]:
df.head(10)

,dew,humidity,pressure,temperature,timestamp,volt,humi,pres,temp
1731000619,24.83318,98.48,1011.58,25.09,1731000619,4.05,NaN,NaN,NaN
1731000678,24.83488,98.49,1011.57,25.09,1731000678,4.05,NaN,NaN,NaN
1731000739,24.83318,98.48,1011.56,25.09,1731000739,4.05,NaN,NaN,NaN
1731000798,24.83148,98.47,1011.51,25.09,1731000798,4.05,NaN,NaN,NaN
1731000859,24.79155,98.47,1011.54,25.05,1731000859,4.05,NaN,NaN,NaN
1731000918,24.78985,98.46,1011.56,25.05,1731000918,4.05,NaN,NaN,NaN
1731000979,24.81979,98.46,1011.52,25.08,1731000979,4.05,NaN,NaN,NaN
1731001039,24.80153,98.47,1011.54,25.06,1731001039,4.05,NaN,NaN,NaN
1731001098,24.77647,98.44,1011.52,25.04,1731001098,4.05,NaN,NaN,NaN
1731001159,24.78305,98.42,1011.54,25.05,1731001159,4.05,NaN,NaN,NaN


In [5]:
# Input readable date
readable_date = "30-03-2025 05:57:00"  # Format: DD-MM-YYYY HH:MM:SS

# Convert readable date to Unix timestamp
start_timestamp = int(datetime.strptime(readable_date, "%d-%m-%Y %H:%M:%S").timestamp())
print(start_timestamp)
end_timestamp = int(datetime.now().timestamp())  # Current time in Unix timestamp

# Filter data berdasarkan rentang waktu dengan validasi kunci
filtered_data = {
    key: value
    for key, value in source_data.items()
    if key.isdigit() and start_timestamp <= int(key) <= end_timestamp
}

1743314220


In [6]:
# Konfigurasi database tujuan
dest_cred = credentials.Certificate("/work/staklimjerukagung-firebase-admin.json")
dest_app = firebase_admin.initialize_app(dest_cred, {
    'databaseURL': 'https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/'
}, name='destination')

In [7]:
# Referensi ke data tujuan
dest_ref = db.reference('/auto_weather_stat/id-03/data', app=dest_app)

In [8]:
# Memindahkan data yang telah difilter ke database tujuan
if filtered_data:
    dest_ref.update(filtered_data)
    print("Data berhasil dipindahkan ke /auto_weather_stat/id-03 di database tujuan.")
else:
    print("Tidak ada data dalam rentang waktu yang ditentukan.")

Data berhasil dipindahkan ke /auto_weather_stat/id-03 di database tujuan.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>